# Test: Parse SnowPilot Data into Data Structures

This notebook demonstrates how to use the `snowpyt_mechparams.snowpilot_utils` module to:
1. Parse SnowPilot/CAAML XML files
2. Convert them to `Layer` and `Slab` objects
3. Use ECTP failure layer as the weak layer definition

In [10]:
import os
import sys

# Import SnowPilot parsing utilities
from snowpyt_mechparams.snowpilot import (
    parse_caaml_file,
    parse_caaml_directory,
)

# Import data structures for inspection
from snowpyt_mechparams.data_structures import Layer, Slab, Pit

print("✅ Imports successful!")

✅ Imports successful!


## Step 1: Parse SnowPilot Files

First, let's parse all the CAAML XML files from the `data/` directory.

In [11]:
# Parse all CAAML files in the data directory
profiles = parse_caaml_directory('data')

print(f"✅ Successfully parsed {len(profiles)} SnowPilot profiles")
print(f"\nFirst profile type: {type(profiles[0])}")

✅ Successfully parsed 50278 SnowPilot profiles

First profile type: <class 'snowpylot.snow_pit.SnowPit'>


## Step 2: Convert to Layer Objects

Let's take one profile and convert it to `Layer` objects.

In [ ]:
# Convert the first profile to layers
test_profile = profiles[0]
layers = caaml_to_layers(test_profile)

print(f"✅ Converted profile to {len(layers)} Layer objects")
print(f"\nLayer type: {type(layers[0])}")
print(f"Layer is instance of Layer class: {isinstance(layers[0], Layer)}")

# Inspect the first layer
if layers:
    print(f"\n{'='*60}")
    print("First Layer Details:")
    print(f"{'='*60}")
    layer = layers[0]
    print(f"  Depth (top):        {layer.depth_top} cm")
    print(f"  Thickness:          {layer.thickness} cm")
    print(f"  Depth (bottom):     {layer.depth_bottom} cm")
    print(f"  Density (measured): {layer.density_measured} kg/m³")
    print(f"  Hand hardness:      {layer.hand_hardness}")
    print(f"  Grain form:         {layer.grain_form}")
    print(f"  Main grain form:    {layer.main_grain_form}")
    print(f"  Grain size:         {layer.grain_size_avg} mm")

✅ Converted profile to 7 Layer objects

Layer type: <class 'snowpyt_mechparams.data_structures.data_structures.Layer'>
Layer is instance of Layer class: True

First Layer Details:
  Depth (top):        0.0 cm
  Thickness:          30.0 cm
  Depth (bottom):     30.0 cm
  Density (measured): None kg/m³
  Hand hardness:      None
  Grain form:         None
  Main grain form:    None
  Grain size:         None mm


## Step 3: Convert to Slab with ECTP Failure Layer

Now let's convert profiles to `Slab` objects using the ECTP failure layer as the weak layer definition.

In [ ]:
# Convert using ECTP failure layer as weak layer definition
slab = caaml_to_slab(test_profile, weak_layer_def="ECTP_failure_layer")

if slab:
    print(f"✅ Successfully created Slab object")
    print(f"\nSlab type: {type(slab)}")
    print(f"Slab is instance of Slab class: {isinstance(slab, Slab)}")
    
    print(f"\n{'='*60}")
    print("Slab Details:")
    print(f"{'='*60}")
    print(f"  Number of layers:    {len(slab.layers)}")
    print(f"  Total thickness:     {slab.total_thickness} cm")
    print(f"  Slope angle:         {slab.angle}°")
    print(f"  Has weak layer:      {slab.weak_layer is not None}")
else:
    print("⚠️  No ECTP failure layer found in this profile")
    print("   Trying next profile...")

✅ Successfully created Slab object

Slab type: <class 'snowpyt_mechparams.data_structures.data_structures.Slab'>
Slab is instance of Slab class: True

Slab Details:
  Number of layers:    6
  Total thickness:     110.0 cm
  Slope angle:         38.0°
  Has weak layer:      False


## Step 4: Find Profiles with ECTP Failure Layers

Let's scan through the profiles to find ones that have ECTP failure layers.

In [ ]:
# Find profiles with ECTP failure layers
slabs_with_ectp = []
profiles_with_ectp = []

for i, profile in enumerate(profiles[:100]):  # Check first 100 profiles
    slab = caaml_to_slab(profile, weak_layer_def="ECTP_failure_layer")
    if slab:
        slabs_with_ectp.append(slab)
        profiles_with_ectp.append((i, profile))

print(f"✅ Found {len(slabs_with_ectp)} profiles with ECTP failure layers (out of {min(100, len(profiles))} checked)")

if slabs_with_ectp:
    print(f"\n{'='*60}")
    print("Example Slab with ECTP Failure Layer:")
    print(f"{'='*60}")
    example_slab = slabs_with_ectp[0]
    print(f"  Profile index:       {profiles_with_ectp[0][0]}")
    print(f"  Number of layers:    {len(example_slab.layers)}")
    print(f"  Total thickness:     {example_slab.total_thickness:.2f} cm")
    print(f"  Slope angle:         {example_slab.angle:.1f}°")
    print(f"\n  Layer details:")
    for idx, layer in enumerate(example_slab.layers):
        print(f"    Layer {idx+1}: {layer.depth_top:.1f}-{layer.depth_bottom:.1f} cm, "
              f"grain: {layer.grain_form}, hardness: {layer.hand_hardness}")

✅ Found 23 profiles with ECTP failure layers (out of 100 checked)

Example Slab with ECTP Failure Layer:
  Profile index:       0
  Number of layers:    6
  Total thickness:     110.00 cm
  Slope angle:         38.0°

  Layer details:
    Layer 1: 0.0-30.0 cm, grain: None, hardness: None
    Layer 2: 30.0-40.0 cm, grain: PP, hardness: None
    Layer 3: 40.0-60.0 cm, grain: None, hardness: None
    Layer 4: 60.0-70.0 cm, grain: DF, hardness: F+
    Layer 5: 70.0-105.0 cm, grain: None, hardness: 1F
    Layer 6: 105.0-110.0 cm, grain: FCxr, hardness: F+


## Step 5: Compare Different Weak Layer Definitions

Let's compare results using different weak layer definitions for profiles that have them.

In [ ]:
# Pick one profile and try different weak layer definitions
if profiles_with_ectp:
    test_profile = profiles_with_ectp[0][1]
    
    print(f"{'='*60}")
    print("Comparing Weak Layer Definitions:")
    print(f"{'='*60}")
    
    definitions = [
        ("All layers (no weak layer)", None),
        ("Layer of concern", "layer_of_concern"),
        ("CT failure layer", "CT_failure_layer"),
        ("ECTP failure layer", "ECTP_failure_layer"),
    ]
    
    for name, definition in definitions:
        slab = caaml_to_slab(test_profile, weak_layer_def=definition)
        if slab:
            print(f"\n{name}:")
            print(f"  ✅ Layers: {len(slab.layers)}, Total thickness: {slab.total_thickness:.2f} cm")
        else:
            print(f"\n{name}:")
            print(f"  ⚠️  No slab created (weak layer not found or no layers above it)")
else:
    print("No profiles with ECTP failure layers found in the first 100 profiles.")

Comparing Weak Layer Definitions:

All layers (no weak layer):
  ✅ Layers: 7, Total thickness: 150.00 cm

Layer of concern:
  ✅ Layers: 5, Total thickness: 105.00 cm

CT failure layer:
  ⚠️  No slab created (weak layer not found or no layers above it)

ECTP failure layer:
  ✅ Layers: 6, Total thickness: 110.00 cm


## Step 6: Statistics on Data Structures

Let's gather some statistics about the parsed data.

In [ ]:
import numpy as np

# Gather statistics from all parsed slabs with ECTP
if slabs_with_ectp:
    num_layers = [len(slab.layers) for slab in slabs_with_ectp]
    thicknesses = [slab.total_thickness for slab in slabs_with_ectp if slab.total_thickness is not None]
    angles = [slab.angle for slab in slabs_with_ectp if not np.isnan(slab.angle)]
    
    print(f"{'='*60}")
    print("Statistics for Slabs with ECTP Failure Layers:")
    print(f"{'='*60}")
    print(f"\nNumber of slabs: {len(slabs_with_ectp)}")
    
    print(f"\nLayers per slab:")
    print(f"  Min:    {min(num_layers)}")
    print(f"  Max:    {max(num_layers)}")
    print(f"  Mean:   {np.mean(num_layers):.1f}")
    print(f"  Median: {np.median(num_layers):.1f}")
    
    if thicknesses:
        print(f"\nSlab thickness (cm):")
        print(f"  Min:    {min(thicknesses):.1f}")
        print(f"  Max:    {max(thicknesses):.1f}")
        print(f"  Mean:   {np.mean(thicknesses):.1f}")
        print(f"  Median: {np.median(thicknesses):.1f}")
    
    if angles:
        print(f"\nSlope angle (degrees):")
        print(f"  Min:    {min(angles):.1f}")
        print(f"  Max:    {max(angles):.1f}")
        print(f"  Mean:   {np.mean(angles):.1f}")
        print(f"  Median: {np.median(angles):.1f}")
        print(f"  Slabs with angle data: {len(angles)}/{len(slabs_with_ectp)}")
else:
    print("No slabs with ECTP failure layers to analyze.")

Statistics for Slabs with ECTP Failure Layers:

Number of slabs: 23

Layers per slab:
  Min:    1
  Max:    8
  Mean:   3.7
  Median: 4.0

Slab thickness (cm):
  Min:    14.0
  Max:    110.0
  Mean:   46.3
  Median: 42.0

Slope angle (degrees):
  Min:    18.0
  Max:    47.0
  Mean:   28.6
  Median: 28.0
  Slabs with angle data: 21/23


## Step 7: Verify Data Structure Properties

Let's verify that the computed properties work correctly.

In [ ]:
if slabs_with_ectp:
    example_slab = slabs_with_ectp[0]
    
    print(f"{'='*60}")
    print("Testing Data Structure Properties:")
    print(f"{'='*60}")
    
    # Test Layer properties
    print("\nLayer Properties:")
    for idx, layer in enumerate(example_slab.layers[:3]):  # First 3 layers
        print(f"\n  Layer {idx+1}:")
        print(f"    depth_top:           {layer.depth_top}")
        print(f"    thickness:           {layer.thickness}")
        print(f"    depth_bottom:        {layer.depth_bottom} (computed)")
        print(f"    hand_hardness:       {layer.hand_hardness}")
        print(f"    hand_hardness_index: {layer.hand_hardness_index} (computed)")
        print(f"    grain_form:          {layer.grain_form}")
        print(f"    main_grain_form:     {layer.main_grain_form} (computed)")
        
        # Verify depth_bottom calculation
        if layer.depth_top is not None and layer.thickness is not None:
            expected_bottom = layer.depth_top + layer.thickness
            if abs(layer.depth_bottom - expected_bottom) < 0.01:
                print(f"    ✅ depth_bottom correctly computed")
            else:
                print(f"    ❌ depth_bottom calculation error")
    
    # Test Slab properties
    print(f"\nSlab Properties:")
    print(f"  Number of layers:  {len(example_slab.layers)}")
    print(f"  Total thickness:   {example_slab.total_thickness} cm")
    
    # Verify total thickness calculation
    manual_total = sum(layer.thickness for layer in example_slab.layers if layer.thickness is not None)
    if abs(example_slab.total_thickness - manual_total) < 0.01:
        print(f"  ✅ total_thickness correctly computed ({manual_total:.2f} cm)")
    else:
        print(f"  ❌ total_thickness calculation error")
    
    print(f"\n✅ Data structure testing complete!")
else:
    print("No slabs available for property testing.")

Testing Data Structure Properties:

Layer Properties:

  Layer 1:
    depth_top:           0.0
    thickness:           30.0
    depth_bottom:        30.0 (computed)
    hand_hardness:       None
    hand_hardness_index: None (computed)
    grain_form:          None
    main_grain_form:     None (computed)
    ✅ depth_bottom correctly computed

  Layer 2:
    depth_top:           30.0
    thickness:           10.0
    depth_bottom:        40.0 (computed)
    hand_hardness:       None
    hand_hardness_index: None (computed)
    grain_form:          PP
    main_grain_form:     PP (computed)
    ✅ depth_bottom correctly computed

  Layer 3:
    depth_top:           40.0
    thickness:           20.0
    depth_bottom:        60.0 (computed)
    hand_hardness:       None
    hand_hardness_index: None (computed)
    grain_form:          None
    main_grain_form:     None (computed)
    ✅ depth_bottom correctly computed

Slab Properties:
  Number of layers:  6
  Total thickness:   110.0 cm
 

## Summary

This notebook demonstrated:

1. ✅ Parsing SnowPilot/CAAML XML files using `parse_caaml_directory()`
2. ✅ Converting to `Layer` objects using `caaml_to_layers()`
3. ✅ Converting to `Slab` objects using `caaml_to_slab()` with `weak_layer_def="ECTP_failure_layer"`
4. ✅ Accessing computed properties like `depth_bottom`, `hand_hardness_index`, `main_grain_form`, and `total_thickness`
5. ✅ Comparing different weak layer definitions

The `snowpyt_mechparams.snowpilot_utils` module successfully converts SnowPilot data into the library's data structures!